# Proyek Analisis Data: [E-Commerce Public Dataset]
- **Nama:** [Muhammad Ikhlas Naufalsyah Ranau]
- **Email:** [naufalsyah.ranau@gmail.com]
- **ID Dicoding:** [mikhlasnr]

## Background E-Commerce Public Dataset



### **Tentang Dataset**
Dataset ini adalah **Brazilian E-Commerce Public Dataset** yang disediakan oleh **Olist**, marketplace terbesar di Brasil. Dataset ini berisi informasi dari **100.000 pesanan** yang dilakukan dari **2016 hingga 2018** di berbagai marketplace di Brasil.


### **Tentang Olist**
Olist adalah department store terbesar di marketplace Brasil yang:
- Menghubungkan bisnis kecil dari seluruh Brasil
- Menyediakan akses ke berbagai channel dengan kontrak tunggal
- Memungkinkan merchant menjual produk melalui Olist Store
- Menggunakan partner logistik untuk pengiriman langsung ke pelanggan


### **Karakteristik Dataset**
- **Sumber**: Data komersial nyata yang telah dianonimkan
- **Periode**: 2016-2018 (3 tahun data)
- **Volume**: 100k+ pesanan
- **Cakupan**: Multiple marketplace di Brasil
- **Anonimisasi**: Referensi perusahaan dan partner diganti dengan nama rumah besar Game of Thrones

### **Fitur Dataset**
Dataset memungkinkan analisis pesanan dari berbagai dimensi:
- **Status pesanan** dan timeline pengiriman
- **Harga** dan performa pembayaran
- **Lokasi pelanggan** dan data geografis
- **Atribut produk** dan kategori
- **Review pelanggan** dan rating kepuasan

### **Catatan Penting**
- Satu pesanan bisa memiliki **multiple items**
- Setiap item bisa dipenuhi oleh **seller yang berbeda**
- Semua teks yang mengidentifikasi toko diganti dengan nama rumah Game of Thrones
- Dataset geolokasi tersedia untuk mapping koordinat lat/lng


### Data Schema

![image info](https://i.imgur.com/HRhd2Y0.png)




## Menentukan Pertanyaan Bisnis

**1. Analisis Pelanggan (Customer Analysis)**
- Bagaimana segmentasi pelanggan berdasarkan nilai dan frekuensi pembelian?
- Wilayah mana yang memiliki pelanggan paling berharga?
- Berapa lama siklus hidup pelanggan dan tingkat retensi?

**2. Analisis Produk (Product Analysis)**
- Kategori produk mana yang paling menguntungkan?
- Bagaimana performa produk berdasarkan review dan rating?
- Apakah ada korelasi antara dimensi produk dan biaya pengiriman?

**3. Analisis Penjual (Seller Analysis)**
- Penjual mana yang memiliki performa terbaik?
- Bagaimana distribusi geografis penjual mempengaruhi penjualan?
- Apakah ada pola dalam rating penjual berdasarkan lokasi?

**4. Analisis Operasional (Operational Analysis)**
- Berapa rata-rata waktu pengiriman dan akurasi estimasi?
- Metode pembayaran mana yang paling populer?
- Bagaimana performa pengiriman berdasarkan wilayah?

## Import Semua Packages/Library yang Digunakan

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Wrangling

### Gathering Data

#### Import Semua Dataset

In [2]:
base_url = 'data'
customers_df = pd.read_csv(base_url + "/customers_dataset.csv")

geolocation_df = pd.read_csv(base_url +  "/geolocation_dataset.csv")

order_items_df = pd.read_csv(base_url + "/order_items_dataset.csv")
order_payments_df = pd.read_csv(base_url + "/order_payments_dataset.csv")
order_reviews_df = pd.read_csv(base_url + "/order_reviews_dataset.csv")
orders_df = pd.read_csv(base_url + "/orders_dataset.csv")

product_category_df = pd.read_csv(base_url + "/product_category_name_translation.csv")
products_df = pd.read_csv(base_url + "/products_dataset.csv")


sellers_df = pd.read_csv(base_url + "/sellers_dataset.csv")

### Data Understanding

#### customers_df

Dataset ini berisi informasi tentang identitas dan lokasi geografis pelanggan. 

Berikut adalah penjelasan detail untuk setiap kolom pada dataset customers:

- **customer_id**: ID unik untuk setiap transaksi pesanan. Format: string alfanumerik 32 karakter. Contoh: "06b8999e2fba1a1fbc88172c00ba8bc7". Setiap pesanan memiliki customer_id yang berbeda, bahkan jika berasal dari pelanggan yang sama.

- **customer_unique_id**: ID permanen untuk setiap pelanggan. Format: string alfanumerik 32 karakter. Contoh: "861eff4711a542e4b93843c6dd7febb0". Digunakan untuk melacak pembelian berulang dari pelanggan yang sama. Satu customer_unique_id dapat memiliki multiple customer_id.

- **customer_zip_code_prefix**: Prefix kode pos pelanggan (5 digit pertama). Format: string numerik. Contoh: "14409", "9790". Mengidentifikasi wilayah geografis tempat tinggal pelanggan.

- **customer_city**: Nama kota tempat tinggal pelanggan. Format: string lowercase. Contoh: "franca", "sao bernardo do campo", "sao paulo". Dapat digunakan untuk analisis geografis dan segmentasi pelanggan.

- **customer_state**: Kode negara bagian tempat tinggal pelanggan. Format: string 2 karakter. Contoh: "SP", "RJ", "MG". Menggunakan singkatan standar negara bagian Brasil.

**Catatan Penting:**
- Satu pelanggan (customer_unique_id) dapat melakukan multiple pesanan, sehingga akan memiliki multiple customer_id
- customer_unique_id adalah kunci untuk analisis customer lifetime value (CLV) dan customer retention
- Dataset ini dapat di-join dengan tabel orders menggunakan customer_id untuk mendapatkan riwayat pesanan pelanggan
- Data geografis (zip_code_prefix, city, state) dapat di-join dengan tabel geolocation untuk mendapatkan koordinat lat/lng
- Dapat digunakan untuk analisis: segmentasi pelanggan, distribusi geografis, customer behavior, dan market penetration


In [3]:
customers_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


#### geolocation_df

Dataset ini berisi informasi kode pos Brasil beserta koordinat geografisnya (latitude dan longitude). Dataset ini digunakan untuk membuat peta dan menghitung jarak antara penjual dan pelanggan.

Berikut adalah penjelasan detail untuk setiap kolom pada dataset geolocation:

- **geolocation_zip_code_prefix**: Prefix kode pos Brasil (5 digit pertama). Kode pos Brasil memiliki format 8 digit (contoh: 01037-000), dan kolom ini menyimpan 5 digit pertama yang mengidentifikasi wilayah geografis tertentu. Contoh: "01037" mengidentifikasi wilayah tertentu di São Paulo.

- **geolocation_lat**: Latitude (garis lintang) dalam format decimal degrees. Nilai ini menunjukkan posisi geografis di utara atau selatan garis khatulistiwa. Untuk Brasil, nilai latitude biasanya negatif karena berada di belahan selatan bumi. Contoh: -23.545621 menunjukkan lokasi sekitar 23.5° di selatan khatulistiwa.

- **geolocation_lng**: Longitude (garis bujur) dalam format decimal degrees. Nilai ini menunjukkan posisi geografis di barat atau timur meridian utama (Greenwich). Untuk Brasil, nilai longitude biasanya negatif karena berada di belahan barat bumi. Contoh: -46.639292 menunjukkan lokasi sekitar 46.6° di barat meridian utama.

- **geolocation_city**: Nama kota dalam format lowercase (huruf kecil). Menunjukkan kota tempat lokasi geografis tersebut berada. Contoh: "sao paulo", "rio de janeiro", "belo horizonte".

- **geolocation_state**: Kode negara bagian Brasil (2 karakter). Brasil memiliki 26 negara bagian dan 1 distrik federal. Kode ini menggunakan singkatan standar. Contoh: "SP" untuk São Paulo, "RJ" untuk Rio de Janeiro, "MG" untuk Minas Gerais.


Catatan Penting:
- Satu kode pos prefix dapat memiliki beberapa koordinat lat/lng berbeda (karena kode pos bisa mencakup area yang luas)
- Koordinat lat/lng dapat digunakan untuk menghitung jarak antara lokasi menggunakan rumus Haversine atau metode geografis lainnya
- Dataset ini dapat di-join dengan tabel customers dan sellers menggunakan zip_code_prefix untuk mendapatkan koordinat geografis mereka



In [4]:
geolocation_df.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


#### order_items_df

Dataset ini berisi informasi detail tentang item-item yang dibeli dalam setiap pesanan. Setiap baris merepresentasikan satu item dalam pesanan tertentu.

Berikut adalah penjelasan detail untuk setiap kolom pada dataset order_items:

- **order_id**: ID unik untuk setiap pesanan. Satu pesanan dapat memiliki beberapa item, sehingga order_id yang sama dapat muncul beberapa kali dalam dataset. Format: string alfanumerik 32 karakter. Contoh: "00010242fe8c5a6d1ba2dd792cb16214"

- **order_item_id**: Nomor urut item dalam satu pesanan. Dimulai dari 1 untuk item pertama dalam pesanan, 2 untuk item kedua, dan seterusnya. Tipe data: integer. Contoh: 1, 2, 3 (menunjukkan item ke-1, ke-2, ke-3 dalam pesanan yang sama)

- **product_id**: ID unik untuk setiap produk. Format: string alfanumerik 32 karakter. Contoh: "4244733e06e7ecb4970a6e2683c13e61". Dapat di-join dengan tabel products untuk mendapatkan informasi detail produk.

- **seller_id**: ID unik untuk penjual yang menjual produk tersebut. Format: string alfanumerik 32 karakter. Contoh: "48436dade18ac8b2bce089ec2a041202". Dapat di-join dengan tabel sellers untuk mendapatkan informasi penjual.

- **shipping_limit_date**: Batas waktu pengiriman yang ditetapkan untuk item tersebut. Format: datetime (YYYY-MM-DD HH:MM:SS). Contoh: "2017-09-19 09:45:35". Ini adalah deadline yang harus dipenuhi penjual untuk mengirimkan item.

- **price**: Harga produk per unit dalam mata uang Real Brasil (BRL). Tipe data: float. Contoh: 58.90, 239.90, 199.00. Harga ini adalah harga yang dibayar pelanggan untuk satu unit produk.

- **freight_value**: Biaya pengiriman (ongkos kirim) untuk item tersebut dalam mata uang Real Brasil (BRL). Tipe data: float. Contoh: 13.29, 19.93, 17.87. Biaya ini terpisah dari harga produk dan biasanya dibebankan kepada pelanggan.

**Catatan Penting:**
- Satu pesanan (order_id) dapat memiliki multiple items dengan order_item_id yang berbeda
- Setiap item dalam pesanan dapat berasal dari seller yang berbeda (seller_id berbeda)
- Total nilai pesanan = sum(price) + sum(freight_value) untuk semua item dalam pesanan tersebut
- Dataset ini adalah tabel fact utama yang menghubungkan orders, products, dan sellers
- Dapat digunakan untuk analisis: nilai pesanan, performa penjual, popularitas produk, dan efisiensi pengiriman

In [5]:
order_items_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


#### order_payments_df

Dataset ini berisi informasi tentang metode pembayaran yang digunakan untuk setiap pesanan. Satu pesanan dapat memiliki multiple metode pembayaran (misalnya: sebagian dibayar dengan kartu kredit, sebagian dengan voucher).

Berikut adalah penjelasan detail untuk setiap kolom pada dataset order_payments:

- **order_id**: ID unik untuk setiap pesanan. Format: string alfanumerik 32 karakter. Contoh: "b81ef226f3fe1789b1e8b2acac839d17". Dapat di-join dengan tabel orders untuk mendapatkan informasi detail pesanan.

- **payment_sequential**: Nomor urut metode pembayaran dalam satu pesanan. Dimulai dari 1 untuk pembayaran pertama, 2 untuk pembayaran kedua, dan seterusnya. Tipe data: integer. Contoh: 1, 2, 3 (menunjukkan pembayaran ke-1, ke-2, ke-3 dalam pesanan yang sama).

- **payment_type**: Jenis metode pembayaran yang digunakan. Format: string. Pilihan yang tersedia:
  - "credit_card": Kartu kredit
  - "debit_card": Kartu debit
  - "boleto": Voucher bank (metode pembayaran populer di Brasil)
  - "voucher": Voucher/gift card
  - "not_defined": Tidak didefinisikan

- **payment_installments**: Jumlah cicilan untuk pembayaran tersebut. Tipe data: integer. Contoh: 1 (lunas), 2, 3, 6, 8, 12. Nilai 1 berarti pembayaran lunas tanpa cicilan.

- **payment_value**: Nilai pembayaran untuk metode pembayaran tersebut dalam mata uang Real Brasil (BRL). Tipe data: float. Contoh: 99.33, 24.39, 65.71. Total payment_value untuk satu pesanan harus sama dengan total nilai pesanan.

**Catatan Penting:**
- Satu pesanan (order_id) dapat memiliki multiple metode pembayaran dengan payment_sequential yang berbeda
- Total payment_value untuk semua metode pembayaran dalam satu pesanan harus sama dengan total nilai pesanan
- payment_installments > 1 hanya berlaku untuk credit_card dan debit_card
- boleto biasanya tidak memiliki cicilan (payment_installments = 1)
- Dataset ini dapat di-join dengan tabel orders menggunakan order_id
- Dapat digunakan untuk analisis: preferensi metode pembayaran, analisis cicilan, conversion rate per metode pembayaran, dan customer payment behavior

In [6]:
order_payments_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


#### order_reviews_df

Dataset ini berisi informasi tentang review dan rating yang diberikan oleh pelanggan setelah menerima produk. Setelah pelanggan membeli produk dari Olist Store, penjual akan diberitahu untuk memenuhi pesanan tersebut. Setelah pelanggan menerima produk, atau pada tanggal pengiriman yang diperkirakan, pelanggan akan menerima survei kepuasan melalui email di mana mereka dapat memberikan rating untuk pengalaman pembelian dan menulis komentar.

Berikut adalah penjelasan detail untuk setiap kolom pada dataset order_reviews:

- **review_id**: ID unik untuk setiap review. Format: string alfanumerik 32 karakter. Contoh: "7bc2406110b926393aa56f80a40eba40". Setiap review memiliki ID yang unik.

- **order_id**: ID unik untuk pesanan yang direview. Format: string alfanumerik 32 karakter. Contoh: "73fc7af87114b39712e6da79b0a377eb". Dapat di-join dengan tabel orders untuk mendapatkan informasi detail pesanan.

- **review_score**: Rating kepuasan pelanggan dalam skala 1-5. Tipe data: integer. Nilai:
  - 1: Sangat tidak puas
  - 2: Tidak puas
  - 3: Netral
  - 4: Puas
  - 5: Sangat puas

- **review_comment_title**: Judul komentar review (opsional). Format: string. Bisa kosong (null) jika pelanggan tidak memberikan judul. Contoh: "Excellent product", "Fast delivery".

- **review_comment_message**: Pesan komentar review (opsional). Format: string. Bisa kosong (null) jika pelanggan tidak memberikan komentar. Contoh: "Recebi bem antes do prazo estipulado" (diterima jauh sebelum batas waktu yang ditetapkan).

- **review_creation_date**: Tanggal review dibuat. Format: datetime (YYYY-MM-DD HH:MM:SS). Contoh: "2018-01-18 00:00:00". Biasanya di-set ke 00:00:00 untuk menyamakan waktu.

- **review_answer_timestamp**: Timestamp ketika review dijawab/dikirim. Format: datetime (YYYY-MM-DD HH:MM:SS). Contoh: "2018-01-18 21:46:59". Waktu aktual ketika pelanggan mengirimkan review.

**Catatan Penting:**
- Tidak semua pesanan memiliki review (beberapa pelanggan tidak memberikan review)
- review_score adalah metrik utama untuk mengukur kepuasan pelanggan
- review_comment_title dan review_comment_message dapat kosong (null)
- review_creation_date biasanya berbeda dengan review_answer_timestamp
- Dataset ini dapat di-join dengan tabel orders menggunakan order_id
- Dapat digunakan untuk analisis: customer satisfaction, sentiment analysis, product quality assessment, seller performance, dan customer feedback trends

In [7]:
order_reviews_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


#### orders_df

Dataset ini berisi informasi utama tentang setiap pesanan dalam sistem e-commerce. Ini adalah tabel inti yang menghubungkan pelanggan dengan item yang dibeli, pembayaran, dan review. Dataset ini melacak seluruh siklus hidup pesanan dari pembelian hingga pengiriman.

Berikut adalah penjelasan detail untuk setiap kolom pada dataset orders:

- **order_id**: ID unik untuk setiap pesanan. Format: string alfanumerik 32 karakter. Contoh: "e481f51cbdc54678b7cc49136f2d6af7". Ini adalah primary key yang menghubungkan dengan semua tabel lainnya.

- **customer_id**: ID pelanggan yang melakukan pesanan. Format: string alfanumerik 32 karakter. Contoh: "9ef432eb6251297304e76186b10a928d". Dapat di-join dengan tabel customers untuk mendapatkan informasi detail pelanggan.

- **order_status**: Status terkini dari pesanan. Format: string. Pilihan yang tersedia:
  - "delivered": Pesanan telah dikirim dan diterima pelanggan
  - "invoiced": Pesanan telah dibuat invoice (belum dikirim)
  - "shipped": Pesanan telah dikirim (sedang dalam perjalanan)
  - "canceled": Pesanan dibatalkan
  - "unavailable": Pesanan tidak tersedia

- **order_purchase_timestamp**: Waktu ketika pesanan dibuat/dibeli. Format: datetime (YYYY-MM-DD HH:MM:SS). Contoh: "2017-10-02 10:56:33". Ini adalah waktu aktual ketika pelanggan menyelesaikan checkout.

- **order_approved_at**: Waktu ketika pesanan disetujui oleh sistem. Format: datetime (YYYY-MM-DD HH:MM:SS). Contoh: "2017-10-02 11:07:15". Biasanya terjadi beberapa menit setelah order_purchase_timestamp.

- **order_delivered_carrier_date**: Waktu ketika pesanan diserahkan ke kurir untuk pengiriman. Format: datetime (YYYY-MM-DD HH:MM:SS). Contoh: "2017-10-04 19:55:00". Bisa kosong (null) jika pesanan belum dikirim.

- **order_delivered_customer_date**: Waktu ketika pesanan diterima oleh pelanggan. Format: datetime (YYYY-MM-DD HH:MM:SS). Contoh: "2017-10-10 21:25:13". Bisa kosong (null) jika pesanan belum diterima.

- **order_estimated_delivery_date**: Estimasi waktu pengiriman yang diberikan kepada pelanggan. Format: datetime (YYYY-MM-DD HH:MM:SS). Contoh: "2017-10-18 00:00:00". Biasanya di-set ke 00:00:00 untuk menyamakan waktu.

**Catatan Penting:**
- order_id adalah primary key yang menghubungkan dengan semua tabel lainnya (order_items, order_payments, order_reviews)
- Satu customer_id dapat memiliki multiple order_id (pelanggan dapat melakukan multiple pesanan)
- order_delivered_carrier_date dan order_delivered_customer_date bisa kosong untuk pesanan yang belum dikirim/diterima
- Waktu pengiriman aktual = order_delivered_customer_date - order_delivered_carrier_date
- Dataset ini dapat di-join dengan tabel customers menggunakan customer_id
- Dapat digunakan untuk analisis: order lifecycle, delivery performance, customer behavior, seasonal trends, dan operational efficiency

In [8]:
orders_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


#### product_category_df

Dataset ini berisi tabel terjemahan untuk nama kategori produk dari bahasa Portugis ke bahasa Inggris. Dataset ini membantu dalam analisis kategori produk dengan menyediakan terjemahan yang mudah dipahami untuk audiens internasional.

Berikut adalah penjelasan detail untuk setiap kolom pada dataset product_category:

- **product_category_name**: Nama kategori produk dalam bahasa Portugis. Format: string. Contoh: "beleza_saude", "informatica_acessorios", "automotivo", "cama_mesa_banho". Ini adalah nama kategori asli yang digunakan dalam dataset products.

- **product_category_name_english**: Terjemahan nama kategori produk dalam bahasa Inggris. Format: string. Contoh: "health_beauty", "computers_accessories", "auto", "bed_bath_table". Ini adalah terjemahan yang lebih mudah dipahami untuk analisis internasional.

**Contoh Kategori Populer:**
- "beleza_saude" → "health_beauty" (Kesehatan & Kecantikan)
- "informatica_acessorios" → "computers_accessories" (Aksesori Komputer)
- "automotivo" → "auto" (Otomotif)
- "cama_mesa_banho" → "bed_bath_table" (Tempat Tidur, Meja & Kamar Mandi)
- "moveis_decoracao" → "furniture_decor" (Furnitur & Dekorasi)
- "esporte_lazer" → "sports_leisure" (Olahraga & Rekreasi)
- "perfumaria" → "perfumery" (Parfum)
- "utilidades_domesticas" → "housewares" (Perlengkapan Rumah Tangga)

**Catatan Penting:**
- Dataset ini adalah tabel lookup/reference yang menghubungkan nama kategori Portugis dengan terjemahan Inggris
- Dapat di-join dengan tabel products menggunakan product_category_name
- Berguna untuk analisis kategori yang lebih mudah dipahami oleh audiens internasional
- Dapat digunakan untuk analisis: category performance, market segmentation, dan product categorization

In [9]:
product_category_df.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


#### products_df

Dataset ini berisi informasi detail tentang setiap produk yang dijual di platform e-commerce. Dataset ini mencakup atribut fisik produk, kategori, dan metadata yang berguna untuk analisis produk dan rekomendasi.

Berikut adalah penjelasan detail untuk setiap kolom pada dataset products:

- **product_id**: ID unik untuk setiap produk. Format: string alfanumerik 32 karakter. Contoh: "1e9e8ef04dbcff4541ed26657ea517e5". Ini adalah primary key yang menghubungkan dengan tabel order_items.

- **product_category_name**: Nama kategori produk dalam bahasa Portugis. Format: string. Contoh: "perfumaria", "artes", "esporte_lazer", "bebes". Dapat di-join dengan tabel product_category_name_translation untuk mendapatkan terjemahan bahasa Inggris.

- **product_name_lenght**: Panjang karakter dari nama produk. Tipe data: integer. Contoh: 40, 44, 46. Mengindikasikan seberapa deskriptif nama produk tersebut.

- **product_description_lenght**: Panjang karakter dari deskripsi produk. Tipe data: integer. Contoh: 287, 276, 250. Mengindikasikan seberapa detail deskripsi produk yang diberikan penjual.

- **product_photos_qty**: Jumlah foto yang tersedia untuk produk. Tipe data: integer. Contoh: 1, 4, 2. Semakin banyak foto, semakin baik untuk konversi penjualan.

- **product_weight_g**: Berat produk dalam gram. Tipe data: integer. Contoh: 225, 1000, 154. Digunakan untuk perhitungan biaya pengiriman dan logistik.

- **product_length_cm**: Panjang produk dalam sentimeter. Tipe data: integer. Contoh: 16, 30, 18. Salah satu dimensi fisik produk.

- **product_height_cm**: Tinggi produk dalam sentimeter. Tipe data: integer. Contoh: 10, 18, 9. Salah satu dimensi fisik produk.

- **product_width_cm**: Lebar produk dalam sentimeter. Tipe data: integer. Contoh: 14, 20, 15. Salah satu dimensi fisik produk.

**Catatan Penting:**
- product_id adalah primary key yang menghubungkan dengan tabel order_items
- product_category_name dapat di-join dengan tabel product_category_name_translation untuk mendapatkan terjemahan bahasa Inggris
- Dimensi fisik (length, height, width) dan weight dapat digunakan untuk analisis logistik dan biaya pengiriman
- product_name_lenght dan product_description_lenght dapat digunakan untuk analisis kualitas listing produk
- product_photos_qty dapat digunakan untuk analisis conversion rate (semakin banyak foto, semakin tinggi kemungkinan terjual)
- Dapat digunakan untuk analisis: product performance, category analysis, logistics optimization, dan product recommendation

In [10]:
products_df.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


#### sellers_df

Dataset ini berisi informasi tentang penjual (sellers) yang menjual produk di platform e-commerce. Dataset ini mencakup identitas penjual dan lokasi geografis mereka, yang berguna untuk analisis distribusi penjual dan performa berdasarkan wilayah.

Berikut adalah penjelasan detail untuk setiap kolom pada dataset sellers:

- **seller_id**: ID unik untuk setiap penjual. Format: string alfanumerik 32 karakter. Contoh: "3442f8959a84dea7ee197c632cb2df15". Ini adalah primary key yang menghubungkan dengan tabel order_items.

- **seller_zip_code_prefix**: Prefix kode pos penjual (5 digit pertama). Format: string numerik. Contoh: "13023", "13844", "20031". Mengidentifikasi wilayah geografis tempat penjual beroperasi.

- **seller_city**: Nama kota tempat penjual beroperasi. Format: string lowercase. Contoh: "campinas", "mogi guacu", "rio de janeiro". Dapat digunakan untuk analisis geografis dan segmentasi penjual.

- **seller_state**: Kode negara bagian tempat penjual beroperasi. Format: string 2 karakter. Contoh: "SP", "RJ", "PE". Menggunakan singkatan standar negara bagian Brasil.

**Catatan Penting:**
- seller_id adalah primary key yang menghubungkan dengan tabel order_items
- Data geografis (zip_code_prefix, city, state) dapat di-join dengan tabel geolocation untuk mendapatkan koordinat lat/lng
- Satu penjual dapat menjual multiple produk dan memiliki multiple order_items
- Dataset ini dapat di-join dengan tabel order_items menggunakan seller_id untuk mendapatkan performa penjual
- Dapat digunakan untuk analisis: seller performance, geographic distribution, seller clustering, dan market penetration analysis

In [11]:
sellers_df.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


### Assessing Data

#### Menilai Tabel customers_df

In [12]:
print("Jumlah duplikasi: ", customers_df.duplicated().sum())
customers_df.info()

Jumlah duplikasi:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


**Insight:**
- xxx
- xxx

### Cleaning Data

**Insight:**
- xxx
- xxx

## Exploratory Data Analysis (EDA)

### Explore ...

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2